In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import seaborn as sns
from matplotlib import pyplot as plt
!pip install sweetviz
import sweetviz as sv

Mounted at /content/drive
     |████████████████████████████████| 15.1MB 3.8MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [94]:
ids = list(set(train_err2.user_id))

In [102]:
temp = train_err2[train_err2['user_id'] == 10000]
a = temp.fwver.value_counts()

In [122]:
a = 0
b = 0
c = []

for id in tqdm(ids):
  temp = train_qui[train_qui['user_id'] == id]
  try:
    a = temp.fwver.value_counts().index[1]
    b +=1
    c.append(id)
  except:
    a = 0

In [128]:
temp = train_err2[train_err['user_id'].isin(c)]

In [133]:
temp2 = temp.drop_duplicates(subset = ['user_id'])

In [134]:
temp_tr = temp2[temp2['problems']].fillna(0)
temp_fls = temp2[temp2['problems'] != True].fillna(0)

In [135]:
len(temp_tr)

915

In [136]:
len(temp_fls)

899

In [137]:
len(temp2)

1814

In [139]:
len(temp_tr)/len(temp2)

0.5044101433296582

In [119]:
temp = train_err2[train_err2['user_id'] == 10000]
temp.fwver.value_counts().index[1]

IndexError: ignored

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [6]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data


train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [7]:
print(train_qui3['time'].max())
print(test_qui3['time'].max())
print(train_err2['time'].max())
print(test_err2['time'].max())

2020-11-30 23:40:00
2020-11-30 23:40:00
2020-12-02 18:51:52
2020-12-14 12:01:18


In [8]:
train_qui3['Date'] = train_qui3['time'].dt.dayofyear
test_qui3['Date'] = test_qui3['time'].dt.dayofyear

train_err2['Date'] = train_err2['time'].dt.dayofyear
test_err2['Date'] = test_err2['time'].dt.dayofyear

In [9]:
prob_list = list(set(train_prob.user_id))

In [66]:
train_err2['problems'] = train_err2['user_id'].isin(prob_list)
one_hot = pd.get_dummies(train_err2['errtype'])

In [67]:
temp = train_err2['problems']
temp_n = train_err2[['problems', 'user_id']]

In [68]:
temp2 = pd.concat([temp, one_hot], axis = 1)
temp2_n = pd.concat([temp_n, one_hot], axis = 1)

In [69]:
temp2_tr = temp2[temp2['problems']].fillna(0)
temp2_fls = temp2[temp2['problems'] != True].fillna(0)

In [70]:
trues = pd.DataFrame(temp2_tr.mean()).transpose()
falses = pd.DataFrame(temp2_fls.mean()).transpose()

In [71]:
trues

,problems,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42
0,1.0,0.000529,0.001227,0.001837,0.047854,0.055866,0.003714,0.003956,0.000004,0.000007,0.012781,0.016655,0.017516,0.002156,0.016543,0.084353,0.06767,0.002801,0.000308,0.000073,0.000238,0.000067,0.127127,0.164749,0.002325,0.00014,0.026622,0.03876,0.038809,0.000037,0.11441,0.070925,0.009141,0.016175,0.000976,0.000509,0.00051,0.00031,0.001162,0.043159,0.006429,0.001569


In [86]:
trues.loc[0,3]

0.0018369221618070148

In [73]:
tru = trues.drop('problems', axis = 1)
fls = falses.drop('problems', axis = 1)

In [74]:
tru

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0.000529,0.001227,0.001837,0.047854,0.055866,0.003714,0.003956,0.000004,0.000007,0.012781,0.016655,0.017516,0.002156,0.016543,0.084353,0.06767,0.002801,0.000308,0.000073,0.000238,0.000067,0.127127,0.164749,0.002325,0.00014,0.026622,0.03876,0.038809,0.000037,0.11441,0.070925,0.009141,0.016175,0.000976,0.000509,0.00051,0.00031,0.001162,0.043159,0.006429,0.001569


In [75]:
fls

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0.001955,0.001873,0.001876,0.104128,0.058997,0.002487,0.00245,0.000015,0.000006,0.004061,0.020147,0.020885,0.000723,0.014106,0.107609,0.082072,0.002761,0.000037,0.000012,0.00003,0.000011,0.10045,0.114464,0.00239,0.000014,0.037294,0.027511,0.02786,0.000013,0.12459,0.066064,0.01217,0.004772,0.00031,0.000609,0.000609,0.000244,0.000781,0.044932,0.006174,0.002506


In [76]:
rslt = tru/(tru+fls)

In [83]:
rslt

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42
0,0.21302,0.395728,0.494767,0.314866,0.48637,0.598989,0.617559,0.193365,0.537034,0.758853,0.452552,0.456132,0.748957,0.539754,0.439427,0.451911,0.503634,0.892746,0.856121,0.887471,0.855471,0.55861,0.590047,0.493177,0.909907,0.416519,0.584874,0.582111,0.732887,0.478704,0.517742,0.428931,0.772175,0.758823,0.455251,0.455922,0.558936,0.598114,0.48994,0.51013,0.38499


In [80]:
rslt.iloc[:,1][0]

0.3957282778569861

In [50]:
temps = temp2_n.groupby(by = 'user_id').sum()

In [53]:
temp_tr = temps[temps['problems'] > 0]
temp_fls = temps[temps['problems'] == 0]

In [61]:
tr_avg = pd.DataFrame(temp_tr.mean()).transpose()
fls_avg = pd.DataFrame(temp_fls.mean()).transpose()

In [62]:
tr_avg/(tr_avg + fls_avg)

,problems,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,1.0,0.314229,0.525751,0.623742,0.437559,0.615823,0.716597,0.732156,0.28866,0.662577,0.841948,0.583225,0.586731,0.834719,0.665019,0.570258,0.582596,0.632028,0.933733,0.909688,0.930316,0.909254,0.681769,0.709004,0.622247,0.944742,0.547187,0.70458,0.702209,0.82284,0.608534,0.645059,0.559758,0.851577,0.841926,0.585869,0.586525,0.682056,0.715857,0.619198,0.638051,0.514488


In [64]:
tr_avg

,problems,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,1517.7562,0.8032,1.862,2.788,72.6306,84.7912,5.6374,6.005,0.0056,0.0108,19.3978,25.2782,26.5852,3.2726,25.1082,128.028,102.7062,4.2514,0.4678,0.1108,0.3618,0.1012,192.9472,250.0486,3.5292,0.212,40.406,58.8288,58.9026,0.0562,173.647,107.6476,13.8732,24.5496,1.4812,0.772,0.7748,0.4698,1.7638,65.5052,9.7574,2.381


In [65]:
fls_avg

,problems,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,0.0,1.7529,1.6796,1.6818,93.3597,52.8964,2.2295,2.1968,0.0138,0.0055,3.6414,18.0639,18.7255,0.648,12.6474,96.4811,73.5845,2.4752,0.0332,0.011,0.0271,0.0101,90.0625,102.6272,2.1425,0.0124,33.4371,24.666,24.9793,0.0121,111.7059,59.2326,10.9111,4.2788,0.2781,0.5457,0.5462,0.219,0.7001,40.2852,5.5351,2.2469


In [144]:
a = tr_avg.transpose()
b = fls_avg.transpose()
c = pd.concat([a,b], axis = 1)

In [150]:
c.columns = ['불만고객', '고객']
c.reset_index(inplace= True)

In [151]:
c

,index,불만고객,고객
0,problems,1517.7562,0.0000
1,errtype_1,0.8032,1.7529
2,errtype_2,1.8620,1.6796
3,errtype_3,2.7880,1.6818
4,errtype_4,72.6306,93.3597
5,errtype_5,84.7912,52.8964
6,errtype_6,5.6374,2.2295
7,errtype_7,6.0050,2.1968
8,errtype_8,0.0056,0.0138
9,errtype_9,0.0108,0.0055


In [152]:
c.to_csv("평균_에러타입_횟수.csv", index = False)
!cp 평균_에러타입_횟수.csv "drive/My Drive/"